import pandas as pd
import numpy as np
#!ls /Users/paxton615/Downloads/facebook-v-predicting-check-ins/ 
df = pd.read_csv('/Users/paxton615/Downloads/facebook-v-predicting-check-ins/train.csv')

In [ ]:
#!ls /Users/paxton615/Downloads/facebook-v-predicting-check-ins/ 


In [ ]:
df = pd.read_csv('/Users/paxton615/Downloads/facebook-v-predicting-check-ins/train.csv')

#### In order to go through the whole process asap:
#### 1. choose data from 2<x<2.5,  1<y<1.5
#### 2. save the subset in a new csv

data = df.query("2<x & x<2.5 & 1<y &y<1.5")
data.to_csv('/Users/paxton615/Github_Personal/facebook_predicting_checkins/facebook80000.csv',index=False)

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('/Users/paxton615/Github_Personal/facebook_predicting_checkins/facebook80000.csv')
data.shape

(83197, 6)

In [3]:
data.head()

,row_id,x,y,accuracy,time,place_id
0,112,2.2360,1.3655,66,623174,7663031065
1,180,2.2003,1.2541,65,610195,2358558474
2,367,2.4108,1.3213,74,579667,6644108708
3,874,2.0822,1.1973,320,143566,3229876087
4,1022,2.0160,1.1659,65,207993,3244363975


#### 1. extract time form 'time'
#### 2. filter out place_id appearance that less than 3

In [4]:
data['day'] = pd.to_datetime(data['time'], unit='s').dt.day
data['hour'] = pd.to_datetime(data['time'], unit='s').dt.hour
data['weekday']= pd.to_datetime(data['time'], unit='s').dt.weekday

In [5]:
place_count = data.groupby('place_id')['row_id'].count()

In [6]:
# place_count[place_count>3].index.values

In [7]:
data_final = data[data['place_id'].isin(place_count[place_count>3].index.values)]

In [8]:
data_final.shape

(80910, 9)

In [9]:
data_final.head()

,row_id,x,y,accuracy,time,place_id,day,hour,weekday
0,112,2.2360,1.3655,66,623174,7663031065,8,5,3
2,367,2.4108,1.3213,74,579667,6644108708,7,17,2
3,874,2.0822,1.1973,320,143566,3229876087,2,15,4
4,1022,2.0160,1.1659,65,207993,3244363975,3,9,5
5,1045,2.3859,1.1660,498,503378,6438240873,6,19,1


### Prepare for modeling

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

1. choose columns and prepare train_test_split
2. feature engineer : Standardize data
3. introducing estimator : knn
4. introducing gridsearch
5. results checking

In [15]:
X = data_final[['x','y','accuracy','day','hour','weekday']]

y = data_final['place_id']

x_train, x_test, y_train, y_test = train_test_split(X,y,random_state=17)

In [16]:
transfer = StandardScaler()

x_train = transfer.fit_transform(x_train)

x_test = transfer.transform(x_test)

In [41]:
knn = KNeighborsClassifier()

In [25]:
param_dict = {'n_neighbors':[3,5,7]}
estimator = GridSearchCV(knn, param_grid=param_dict, cv=10)
estimator.fit(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [3, 5, 7]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

above parameters took one minute to calculate

In [39]:
y_predict = estimator.predict(x_test)
print('y_predict:\n',y_predict)
print('true_values VS predict_values:\n', (y_test==y_predict).value_counts())
print('true_values VS predict_values(percentage):\n',(y_test==y_predict).value_counts(normalize=True))

score = estimator.score(x_test, y_test)
print('accuracy: \n', score)

print('best estimator: \n', estimator.best_estimator_)

print('best score: \n', estimator.best_score_)

print('best param: \n', estimator.best_params_)

print('cv_results: \n', estimator.cv_results_)

y_predict:
 [1658077953 5991509451 2998156180 ... 3981920612 9474162554 1314542379]
true_values VS predict_values:
 False    12857
True      7371
Name: place_id, dtype: int64
true_values VS predict_values(percentage):
 False    0.635604
True     0.364396
Name: place_id, dtype: float64
accuracy: 
 0.3643958868894602
best estimator: 
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='uniform')
best score: 
 0.35997495138591346
best param: 
 {'n_neighbors': 7}
cv_results: 
 {'mean_fit_time': array([0.03451529, 0.03302484, 0.03310535]), 'std_fit_time': array([0.00264123, 0.00191954, 0.00238236]), 'mean_score_time': array([0.17495518, 0.18964679, 0.20968122]), 'std_score_time': array([0.00978546, 0.01134367, 0.02261726]), 'param_n_neighbors': masked_array(data=[3, 5, 7],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors'

In [42]:
param_dict = {'n_neighbors':[7,9,11]}
estimator = GridSearchCV(knn, param_grid=param_dict, cv=10)
estimator.fit(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [7, 9, 11]}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

took one minute to calculate

In [43]:
y_predict = estimator.predict(x_test)
print('y_predict:\n',y_predict)
print('true_values VS predict_values:\n', (y_test==y_predict).value_counts())
print('true_values VS predict_values(percentage):\n',(y_test==y_predict).value_counts(normalize=True))

score = estimator.score(x_test, y_test)
print('accuracy: \n', score)

print('best estimator: \n', estimator.best_estimator_)

print('best score: \n', estimator.best_score_)

print('best param: \n', estimator.best_params_)

print('cv_results: \n', estimator.cv_results_)

y_predict:
 [1658077953 5991509451 2998156180 ... 3981920612 9474162554 1314542379]
true_values VS predict_values:
 False    12857
True      7371
Name: place_id, dtype: int64
true_values VS predict_values(percentage):
 False    0.635604
True     0.364396
Name: place_id, dtype: float64
accuracy: 
 0.3643958868894602
best estimator: 
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=7, p=2,
           weights='uniform')
best score: 
 0.35997495138591346
best param: 
 {'n_neighbors': 7}
cv_results: 
 {'mean_fit_time': array([0.036268  , 0.03240385, 0.03279438]), 'std_fit_time': array([0.00443847, 0.00086346, 0.00106345]), 'mean_score_time': array([0.20882046, 0.20893111, 0.21957867]), 'std_score_time': array([0.02220518, 0.00973246, 0.00983256]), 'param_n_neighbors': masked_array(data=[7, 9, 11],
             mask=[False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'n_neighbors

In [44]:
param_dict = {'n_neighbors':[6,7,8,10,20,30]}
estimator = GridSearchCV(knn, param_grid=param_dict, cv=10)
estimator.fit(x_train, y_train)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': [6, 7, 8, 10, 20, 30]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

took 2-3 minutes to calculate

In [45]:
y_predict = estimator.predict(x_test)
print('y_predict:\n',y_predict)
print('true_values VS predict_values:\n', (y_test==y_predict).value_counts())
print('true_values VS predict_values(percentage):\n',(y_test==y_predict).value_counts(normalize=True))

score = estimator.score(x_test, y_test)
print('accuracy: \n', score)

print('best estimator: \n', estimator.best_estimator_)

print('best score: \n', estimator.best_score_)

print('best param: \n', estimator.best_params_)

print('cv_results: \n', estimator.cv_results_)

y_predict:
 [1168869217 5991509451 2998156180 ... 3981920612 9474162554 1314542379]
true_values VS predict_values:
 False    12880
True      7348
Name: place_id, dtype: int64
true_values VS predict_values(percentage):
 False    0.636741
True     0.363259
Name: place_id, dtype: float64
accuracy: 
 0.36325884912003165
best estimator: 
 KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=6, p=2,
           weights='uniform')
best score: 
 0.36017270360238623
best param: 
 {'n_neighbors': 6}
cv_results: 
 {'mean_fit_time': array([0.03402042, 0.03296535, 0.03389199, 0.03262513, 0.03331811,
       0.03493764]), 'std_fit_time': array([0.00119549, 0.00100204, 0.0035345 , 0.00122208, 0.00180442,
       0.00174567]), 'mean_score_time': array([0.19771111, 0.19993072, 0.20526772, 0.21453116, 0.25970049,
       0.29864771]), 'std_score_time': array([0.01298284, 0.00824809, 0.00976168, 0.01133703, 0.01459575,
       0.01544